<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Table of Contents
* [Attaching data to a grid](#Attaching-data-to-a-grid)
  * [Load data from an ESRI ASCII file](#Load-data-from-an-ESRI-ASCII-file)
  * [Load data from OpenTopography](#Load-data-from-OpenTopography)

## Attaching data to a grid

You can attach data to any *Landlab* model grid and at any grid element through *Landlab* data
**fields**. Data are set and accessed in the same way for any grid type.

We'll start by attaching some data to a grid's nodes. Below we see two different methods for
adding data to a grid.

In [ ]:
from landlab import RasterModelGrid

grid = RasterModelGrid((4, 5))
grid.add_zeros("foo", at="node")
grid.at_node["bar"] = [1.0] * grid.number_of_nodes

To access the data, we use the `at_node` data structure, which is dictionary whose keys are
field names, and values are *numpy* arrays.

In [ ]:
print(f"All at-node fields: {list(grid.at_node)!r}")

The values of the field nammed, "foo",

In [ ]:
grid.at_node["foo"]

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
# Get the values for the field names, "bar"
grid.at_node["bar"]
```
</details>

### Load data from an ESRI ASCII file

There are other ways to add fields to grids. Below we show how to read a field from a data file
(in ESRI ASCII format), and then download data from *OpenTopography* and add it to a grid.

The [read_esri_ascii] function creates a new `RasterModelGrid` with some data attached
to its *nodes*.

[read_esri_ascii]: https://landlab.readthedocs.io/en/master/reference/io/esri_ascii.html#landlab.io.esri_ascii.read_esri_ascii

In [ ]:
from landlab.io import read_esri_ascii

(grid, z) = read_esri_ascii("data/square-test-basin.asc", name="topographic__elevation")

In [ ]:
grid.imshow("topographic__elevation")

Use some of the functions we learned about in the previous lesson to get some information about
this DEM. For example,
* How large is this DEM?
* How many rows and columns are there?
* What is the resolution of this DEM?
* What are the maximum and minimum elevation values?

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
# Write your code here
print(f"Number of points = {grid.number_of_nodes}")
print(f"Number of rows and columns = {grid.shape}")
print(f"Resolution = {grid.spacing}")
print(f"Maximum elevation = {grid.at_node['topographic__elevation'].max()}")
print(f"Minimum elevation = {grid.at_node['topographic__elevation'].min()}")
```
</details>

### Load data from OpenTopography

We'll now download some data from *OpenTopography* and add it to a `RasterModelGrid`. To do this,
we use the [bmi_topography] package.

[bmi_topography]: https://bmi-topography.readthedocs.io/en/latest/

In [ ]:
from bmi_topography import Topography

In [ ]:
?Topography

To download *OpenTopography* data, we need to know the bounding box of the data we want.
This is provided through the `north`, `south`, `east`, and `west` keywords (in degrees).

For this example, I've used a bounding box that contains Longs Peak in Colorado.

In [ ]:
lat, lon = (40.2530073213, -105.609067564)  # Longs Peak
height, width = 7.5 / 60.0, 7.5 / 60.0

longs_peak_dem = Topography(
    north=lat + height,
    south=lat - height,
    east=lon + width,
    west=lon - width,
    output_format="GTiff",
    dem_type="SRTMGL3",
)
longs_peak = longs_peak_dem.load()

The elevation data are stored in an *xarray* [*DataArray*](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html). You can access the values as a *numpy* array
through the ``values`` attribute. Notice the shape of the array.

What about the elevation values? Do they seem correct? What are the max and min values? What do you think
the units are?

[DataArray]: https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html

In [ ]:
longs_peak.values.shape

In [ ]:
longs_peak.values.min(), longs_peak.values.max()

How do the above two answers change if you select a different DEM type (`"SRTMGL1"`, say)?

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
lat, lon = (40.2530073213, -105.609067564)  # Longs Peak
height, width = 7.5 / 60.0, 7.5 / 60.0

longs_peak_dem = Topography(
    north=lat + height,
    south=lat - height,
    east=lon + width,
    west=lon - width,
    output_format="GTiff",
    dem_type="SRTMGL1",
)
longs_peak = longs_peak_dem.load()

print(f"Shape of the data = {longs_peak.values.shape}")
print(f"Minimum elevation = {longs_peak.values.min()}")
print(f"Maximum elevation = {longs_peak.values.max()}")
```
</details>

Now create a new ``RasterModelGrid`` and attach your data to it.

In [ ]:
import numpy as np
from landlab import RasterModelGrid

z = np.flipud(longs_peak.values.squeeze())
grid = RasterModelGrid(z.shape, xy_spacing=(70.0, 90.0))
grid.at_node["topographic__elevation"] = z

Notice I've used an ``xy_spacing`` of ``(70.0, 90.0)``. Why's that?

We can create a quick plot of the data field using the ``imshow`` method function of the grid.
Does this look right?

In [ ]:
grid.imshow("topographic__elevation", cmap="terrain", vmin=0)

Some additional things to try:
* Fetch OpenTopography data from another location and another width and height
* Fetch OpenTopography using different ``dem_type`` (the
  [*bmi-topography* documentation](https://bmi-topography.readthedocs.io) has a list of supported types).
* What's the resolution of the dem you downloaded?

In this tutorial we looked at how we can add data to *Landlab* grid *nodes*. In the next tutorial we'll have a look at some of the other grid elements (*links* and *cells*).

👉 [More grid elements](03-links-cells.ipynb)